# Librerías utilizadas

In [75]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import inv,eig
from numpy import dot
import matplotlib.style as style
from scipy import signal
import pandas as pd
style.use('default')

# Funciones

In [76]:
def GaussianNoise(SNR,Ps,M,N):
    """
    Recibe una potencia de señal Ps y una relación señal-ruido deseada
    SNR y retorna un vector Nu correspondiente a muestras del ruido 
    blanco gaussiano a sumarle a la señal
    """
    mean_nu = 0
    var_nu = Ps/(10**(SNR/10))
    std_nu = np.sqrt(var_nu) # sigma_n
    Nu = np.random.normal(mean_nu, std_nu, size=(M,N))
    return (Nu)

def filter(X, Wn, filtertype):
    order = 4
    [b, a] = signal.butter(order, Wn, btype=filtertype)  #coeficientes del filtro butterworth utilizado
    Y = signal.lfilter(b, a, X)  #filtro la señal para limitar el ruido
    return Y

def getS(X):
    X=np.matrix(X)
    M = X.shape[0]
    N = X.shape[1]
    S=np.zeros((M,M),dtype='complex')
    for n in range(0, N):
        S_aux = (1 / N) * (X[:, n] @ X[:, n].conj().T)
        S = S + S_aux
    return S

def print_matrix(list_of_list):
    number_width = len(str(max([max(i) for i in list_of_list])))
    cols = max(map(len, list_of_list))
    output = '+'+('-'*(number_width+2)+'+')*cols + '\n'
    for row in list_of_list:
        for column in row:
            output += '|' + ' {:^{width}d} '.format(column, width = number_width)
        output+='|\n+'+('-'*(number_width+2)+'+')*cols + '\n'
    return output

# Definición de constantes y unidades

In [77]:
kHz = 10 ** 3
MHz = 10 ** 6
GHz = 10 ** 9
c = 3 * 10 ** 8
ms = 10 ** -3

# Definición de variables

In [79]:
M = 8  # Número de sensores
D = 2  # Número de señales
fs = 64 * MHz
N = 1000 # Número de snapshots
T = 2 * ms # Tiempo de toma de muestras

theta_deg = np.array([35,157])  #DOA en grados
theta = theta_deg * np.pi / 180           #DOA en radianes

# Defición de señal y array de sensores
Fc = np.array([1.5* GHz, 1.5* GHz]) # Frecuencia de portadora de la señal transmitida 1

lambda_c = c / Fc
d = lambda_c[0] / 2  #Separación entre sensores

K = 2 * np.pi / (lambda_c)


t = np.random.randint(0,int(T*fs),size=N) * 1/fs # Vector de tiempos de muestreo (tomas de snapshots)

# Definición de señales y muestras

## Genero la matriz F de tamaño (D,1,N)

In [81]:
F = np.zeros((D,1,N))
s_amp = np.array([10, 20])
s_freq = np.array([440,3500])
s1 = s_amp[0] * np.cos(2 * np.pi * s_freq[0] * t) # Vector de señal 1 en el tiempo de toma de snapshots
s2 = s_amp[1] * np.cos(2 * np.pi * s_freq[1] * t) # Vector de señal 2 en el tiempo de toma de snapshots

F[0,0,:] = s1
F[1,0,:] = s2

## Genero la matriz A de tamaño (M,D) considerando $g(\theta)=1 \quad \forall \  \theta$

In [82]:
A=np.asmatrix(np.zeros((M,D),dtype='complex'))
for i in range(0, M):
    for j in range(0, D):
        A[i, j] = np.e ** (-1j * i * K[j] * d * np.cos(theta[j]))

## Genero el vector W de tamaño (M,1,N) de ruido $W\sim N(0,1)$

In [84]:
SNR = 7
mean_nu = 0
Ps = (s_amp[0]**2+s_amp[1]**2)/2
var_nu = Ps/(10**(SNR/10))
std_nu = np.sqrt(var_nu)
W = np.random.normal(mean_nu, std_nu, size=(M,1,N))

## Genero el vector de muestras $X=A \times F + W$ de tamaño (M,1,N)

In [85]:
X = np.zeros((M,1,N),dtype=complex)
X_matrix = np.asmatrix(np.zeros((M,1)),dtype=complex)

for n in range (0,N):
    F_matrix = np.asmatrix(F[:,:,n])
    W_matrix = np.asmatrix(W[:,:,n])
    X_matrix = A @ F_matrix + W_matrix
    X[:,:,n] = X_matrix

# Calculo el vector de covarianza S

In [86]:
S = np.asmatrix(np.zeros((M,M),dtype=complex))

for n in range (0,N):
    X_matrix=np.asmatrix(X[:,:,n])
    S = S + (1/N)*(X_matrix @ X_matrix.H)

# Encuentro autovalores y autovectores S y el autovalor mínimo $\lambda_{min}$ tal que $|S-\lambda_{min}\cdot S_0|=0$

In [87]:
[aval, avec] = eig(S)
S0 = np.identity(M)
p = aval.argsort()
aval=np.abs(aval[p])
avec=avec[:,p]
aval_min = aval[0]

# Encuentro la multiplicidad Q de $\lambda_min$ y el número estimado de señales $\hat{D}$

In [89]:
Q = 0
umbral = 0.5 * aval_min
for i in range (0,M):
    if aval[i]-aval_min<umbral:
        Q += 1
D_est = M - Q

# Formo la matriz de subespacio de ruido $E_N$

In [90]:
EN = np.asmatrix(avec[:,0:Q])

# Evalúo la función $P_{MU}$ para distintas frecuencias de portadora y distintos ángulos de arribo

In [92]:
a=np.asmatrix(np.zeros((M,1),dtype=complex))

fc_est = np.linspace(1 * GHz, 2 * GHz,num=20)
#fc_est = np.array([1.5 *GHz])
K_est = 2 * np.pi * fc_est/c
theta_est = np.linspace(0, np.pi ,num=1000)
P_MU=np.asmatrix(np.zeros((len(theta_est),len(K_est)) , dtype=complex))
for i in range (0, len(K_est)):
    for j in range(0, len(theta_est)):
        for k in range(0,M):
            a[k] = np.e ** (-1j * k * K_est[i] * d * np.cos(theta_est[j]))
            P_MU[j,i] = 1/(a.H @ EN @ EN.H @ a)


In [71]:
plt.figure()
plt.plot(theta_est*180/np.pi,np.abs(np.array(P_MU)))
plt.show()

In [93]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
X, Y = np.meshgrid(K_est*c/(2*np.pi), theta_est*180/np.pi)
ax.plot_surface(X, Y, np.abs(P_MU), cmap='viridis')
ax.set_xlabel(r'$f_{est}$')
ax.set_ylabel(r'$\theta_{est}$')
ax.set_zlabel(r'$P_{MU}$')
plt.show()

In [95]:
idx = np.unravel_index(np.argmax(P_MU, axis=None), P_MU.shape)
print(K_est[idx[1]]*c/(2*np.pi))
print(theta_est[idx[0]]*180/np.pi)

1684210526.3157895
145.04504504504504
